In [1]:
import pandas as pd
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import decomposition, ensemble

In [2]:
data = pd.read_csv('data/data/Youtube_data.csv')
data['Text'] = data['Title']+data['Description']
data = data[['Text','Category']]

In [3]:
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(data['Text'].values.astype('U'))
data_tfidf_ngram =  tfidf_vect_ngram.transform(data['Text'].values.astype('U'))
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(data_tfidf_ngram, data['Category'])

encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [4]:
from keras import layers, models, optimizers

Using TensorFlow backend.


In [5]:
# create input layer 
input_layer = layers.Input((train_x.shape[1], ), sparse=True)

# create hidden layer
hidden_layer1 = layers.Dense(100, activation="relu")(input_layer)

hidden_layer2 = layers.Dense(100, activation="relu")(hidden_layer1)

# create output layer
output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer2)

classifier = models.Model(inputs = input_layer, outputs = output_layer)
classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')

classifier.fit(train_x, train_y)

# predict the labels on validation dataset
predictions = classifier.predict(valid_x)
predictions = predictions.argmax(axis=-1)

metrics.accuracy_score(predictions, valid_y)

Epoch 1/1
2487/2487 [==============================] - 3s 1ms/step - loss: -6.2719


0.16646562123039807